In [29]:
import json 
with open("data/annotations.json", "r") as f:
    annotations = json.load(f)

In [30]:
annotations.keys()

dict_keys(['info', 'images', 'annotations', 'scene_annotations', 'licenses', 'categories', 'scene_categories'])

In [31]:
images = annotations["images"]

In [32]:
from PIL import Image
import numpy as np 

In [ ]:
path = images[0]["file_name"]

# open image from path
img = Image.open("data/" + path)
img_array = np.array(img)

In [ ]:
img.show()

In [ ]:
import os

In [ ]:
batchFolderNames = []
for filename in os.listdir("data"):
    if not filename.startswith('.') and os.path.splitext(filename)[1] != '.csv' and os.path.splitext(filename)[1] != '.json':
        batchFolderNames.append(filename)
print(batchFolderNames)

In [ ]:
import shutil

In [ ]:
def splitData(batchFolderName, trainTestValCumulativeSplit):
    os.makedirs("data/" + batchFolderName + "/train")
    os.makedirs("data/" + batchFolderName + "/test")
    os.makedirs("data/" + batchFolderName + "/val")
    fileNames = []
    for image in images:
        if image['file_name'].split('/')[0] == batchFolderName:
            fileNames.append(image['file_name'])
    fileNames.sort()
    print(fileNames)
    for i, file in enumerate(fileNames):
        file = list(file.split("/"))
        if (i / len(fileNames)) < trainTestValCumulativeSplit[0]:
            shutil.move("data/" + file[0] + '/' + file[1], "data/" + file[0] + '/train/' + file[1])
        if trainTestValCumulativeSplit[0] < (i / len(fileNames)) < trainTestValCumulativeSplit[1]:
            shutil.move("data/" + file[0] + '/' + file[1], "data/" + file[0] + '/test/' + file[1])
        if trainTestValCumulativeSplit[1] < (i / len(fileNames)):
            shutil.move("data/" + file[0] + '/' + file[1], "data/" + file[0] + '/val/' + file[1])

In [ ]:
trainTestValCumulativeSplit = [0.8, 0.9, 1.0]
for batch in batchFolderNames:
    splitData(batch, trainTestValCumulativeSplit)